In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import os

In [4]:

# Create plots directory if it doesn't exist
if not os.path.exists('plots'):
    os.makedirs('plots')

# Load Sierra Leone dataset
df = pd.read_csv('data/sierraleone-bumbuna.csv')
print("First few rows of the Sierra Leone dataset:")
print(df.head())

First few rows of the Sierra Leone dataset:
          Timestamp  GHI  DNI  DHI  ModA  ModB  Tamb    RH   WS  WSgust  \
0  2021-10-30 00:01 -0.7 -0.1 -0.8   0.0   0.0  21.9  99.1  0.0     0.0   
1  2021-10-30 00:02 -0.7 -0.1 -0.8   0.0   0.0  21.9  99.2  0.0     0.0   
2  2021-10-30 00:03 -0.7 -0.1 -0.8   0.0   0.0  21.9  99.2  0.0     0.0   
3  2021-10-30 00:04 -0.7  0.0 -0.8   0.0   0.0  21.9  99.3  0.0     0.0   
4  2021-10-30 00:05 -0.7 -0.1 -0.8   0.0   0.0  21.9  99.3  0.0     0.0   

   WSstdev   WD  WDstdev    BP  Cleaning  Precipitation  TModA  TModB  \
0      0.0  0.0      0.0  1002         0            0.0   22.3   22.6   
1      0.0  0.0      0.0  1002         0            0.0   22.3   22.6   
2      0.0  0.0      0.0  1002         0            0.0   22.3   22.6   
3      0.0  0.0      0.0  1002         0            0.1   22.3   22.6   
4      0.0  0.0      0.0  1002         0            0.0   22.3   22.6   

   Comments  
0       NaN  
1       NaN  
2       NaN  
3       Na

In [5]:

# Summary statistics
print("\nSummary Statistics:")
print(df.describe())

# Missing values report
print("\nMissing Values:")
missing = df.isna().sum()
print(missing)
threshold = len(df) * 0.05
print("\nColumns with >5% Missing:")
print(missing[missing > threshold])


Summary Statistics:
                 GHI            DNI            DHI           ModA  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      201.957515     116.376337     113.720571     206.643095   
std       298.495150     218.652659     158.946032     300.896893   
min       -19.500000      -7.800000     -17.900000       0.000000   
25%        -2.800000      -0.300000      -3.800000       0.000000   
50%         0.300000      -0.100000      -0.100000       3.600000   
75%       362.400000     107.000000     224.700000     359.500000   
max      1499.000000     946.000000     892.000000    1507.000000   

                ModB           Tamb             RH             WS  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      198.114691      26.319394      79.448857       1.146113   
std       288.889073       4.398605      20.520775       1.239248   
min         0.000000      12.300000       9.900000       0.000000   
25%         

In [6]:

# Outlier detection and cleaning
key_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
df_zscores = df[key_columns].apply(zscore, nan_policy='omit')
outliers = (df_zscores.abs() > 3).any(axis=1)
print(f"\nNumber of rows with outliers: {outliers.sum()}")
for col in key_columns:
    df[col].fillna(df[col].median(), inplace=True)
df.dropna(subset=['Timestamp'], inplace=True)
df.to_csv('data/sierraleone_clean.csv', index=False)
print("Cleaned data saved to data/sierraleone_clean.csv")


Number of rows with outliers: 16292


C:\Users\jakin\AppData\Local\Temp\ipykernel_12472\2270888643.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\jakin\AppData\Local\Temp\ipykernel_12472\2270888643.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

Cleaned data saved to data/sierraleone_clean.csv


In [7]:

# Time series analysis
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
plt.figure(figsize=(12, 8))
plt.plot(df['Timestamp'], df['GHI'], label='GHI')
plt.plot(df['Timestamp'], df['DNI'], label='DNI')
plt.plot(df['Timestamp'], df['DHI'], label='DHI')
plt.plot(df['Timestamp'], df['Tamb'], label='Tamb')
plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.title('Sierra Leone: Time Series of GHI, DNI, DHI, and Tamb')
plt.legend()
plt.tight_layout()
plt.savefig('plots/sierraleone_ghi_dni_dhi_tamb_time_series.png')
plt.close()

In [8]:

# Cleaning impact
cleaning_impact = df.groupby('Cleaning')[['ModA', 'ModB']].mean()
print("\nAverage ModA and ModB by Cleaning Flag:")
print(cleaning_impact)
cleaning_impact.plot(kind='bar', title='Sierra Leone: Average ModA and ModB Pre/Post Cleaning')
plt.ylabel('Value')
plt.savefig('plots/sierraleone_cleaning_impact.png')
plt.close()


Average ModA and ModB by Cleaning Flag:
                ModA        ModB
Cleaning                        
0         206.578599  198.038150
1         273.309252  277.231102


In [9]:

# Correlation heatmap
corr_columns = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB']
corr_matrix = df[corr_columns].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Sierra Leone: Correlation Heatmap')
plt.savefig('plots/sierraleone_correlation_heatmap.png')
plt.close()

In [10]:

# Scatter plot: WS vs GHI
plt.figure(figsize=(8, 6))
plt.scatter(df['WS'], df['GHI'], alpha=0.5)
plt.xlabel('Wind Speed (m/s)')
plt.ylabel('GHI (W/m^2)')
plt.title('Sierra Leone: Wind Speed vs GHI')
plt.savefig('plots/sierraleone_ws_vs_ghi.png')
plt.close()

In [11]:

# Wind analysis: GHI histogram
plt.figure(figsize=(8, 6))
plt.hist(df['GHI'], bins=30)
plt.xlabel('GHI (W/m^2)')
plt.ylabel('Frequency')
plt.title('Sierra Leone: GHI Distribution')
plt.savefig('plots/sierraleone_ghi_histogram.png')
plt.close()

# Wind direction distribution
wd_counts = df['WD'].value_counts().sort_index()
plt.figure(figsize=(8, 6))
plt.bar(wd_counts.index, wd_counts.values)
plt.xlabel('Wind Direction (°N)')
plt.ylabel('Count')
plt.title('Sierra Leone: Wind Direction Distribution')
plt.savefig('plots/sierraleone_wind_direction.png')
plt.close()

In [12]:

# Temperature analysis: RH vs Tamb
plt.figure(figsize=(8, 6))
plt.scatter(df['RH'], df['Tamb'], alpha=0.5)
plt.xlabel('Relative Humidity (%)')
plt.ylabel('Ambient Temperature (°C)')
plt.title('Sierra Leone: RH vs Tamb')
plt.savefig('plots/sierraleone_rh_vs_tamb.png')
plt.close()

In [13]:

# Bubble chart: GHI vs Tamb, bubble size = RH
plt.figure(figsize=(10, 8))
plt.scatter(df['Tamb'], df['GHI'], s=df['RH']*10, alpha=0.5)
plt.xlabel('Ambient Temperature (°C)')
plt.ylabel('GHI (W/m^2)')
plt.title('Sierra Leone: GHI vs Tamb (Bubble Size: RH)')
plt.savefig('plots/sierraleone_bubble_ghi_tamb_rh.png')
plt.close()

print("Sierra Leone EDA complete. Plots saved to plots/ directory.")

Sierra Leone EDA complete. Plots saved to plots/ directory.
